## Evaluation

Batch prediction and evaluation are very similar. They are based on DataFlow pipeline and CloudML provides Evaluate and Prediction DataFlow transform. Datalab can generate DataFlow pipeline code template for you, just like Preprocessing.

Run "%ml evaluate" to generate input cell.

In [ ]:
%ml evaluate

After fill in the required fields, run it to generate code.

In [ ]:
%%ml evaluate
preprocessed_eval_data_path: /content/datalab/ml/iris/preprocessed/features_eval-00000-of-00001
metadata_path: /content/datalab/ml/iris/preprocessed/metadata.yaml
model_dir: /content/datalab/ml/iris/model3000/model
output_dir: /content/datalab/ml/iris/evaluate
output_prediction_name: predictions


Run the generated code. Optionally uncomment the code for creating confusion matrix plot. Note that the confusionmatrix code is only generated in classification case. 

In [1]:

# header
"""
Following code is generated from command line:
%%ml evaluate
preprocessed_eval_data_path: /content/datalab/ml/iris/preprocessed/features_eval-00000-of-00001
metadata_path: /content/datalab/ml/iris/preprocessed/metadata.yaml
model_dir: /content/datalab/ml/iris/model3000/model
output_dir: /content/datalab/ml/iris/evaluate
output_prediction_name: predictions

Please modify as appropriate!!!
"""

# imports
import apache_beam as beam
import google.cloud.ml as ml
import google.cloud.ml.analysis as analysis
import google.cloud.ml.io as io
import json
import os

# defines
def extract_values((example, prediction)):
  feature = json.loads(example.values()[0])['features']['feature']
  values = {'target': feature['species']['int64List']['value'][0]}
  values.update(prediction)
  return values

OUTPUT_DIR = '/content/datalab/ml/iris/evaluate'
pipeline = beam.Pipeline('DirectPipelineRunner')


# evaluation
eval_data_source = beam.io.TextFileSource('/content/datalab/ml/iris/preprocessed/features_eval-00000-of-00001', strip_trailing_newlines=True)
eval_features = pipeline | beam.Read('ReadEvalData', eval_data_source)
trained_model = pipeline | io.LoadModel('/content/datalab/ml/iris/model3000/model')
evaluations = (eval_features | ml.Evaluate(trained_model, label='Evaluate')
    | beam.Map('CreateEvaluations', extract_values))
eval_data_sink = beam.io.TextFileSink(os.path.join(OUTPUT_DIR, 'eval'))
evaluations | beam.Write('WriteEval', eval_data_sink)

# analysis
def make_data_for_analysis(values):
  return {
      'target': values['target'],
      'predicted': values['predictions'],
      'score': 0.0,
  }

metadata = pipeline | io.LoadMetadata('/content/datalab/ml/iris/preprocessed/metadata.yaml')
analysis_source = evaluations | beam.Map('CreateAnalysisSource', make_data_for_analysis)
confusion_matrix, precision_recall, logloss = (analysis_source |
    analysis.AnalyzeModel('Analyze Model', metadata))
confusion_matrix_file = os.path.join(OUTPUT_DIR, 'analyze_cm.json')
confusion_matrix_sink = beam.io.TextFileSink(confusion_matrix_file, shard_name_template='')
confusion_matrix | beam.io.Write('WriteConfusionMatrix', confusion_matrix_sink)

# run pipeline
pipeline.run()

# View Confusion Matrix with the following code:
#
import datalab.ml
import yaml
with ml.util._file.open_local_or_gcs(confusion_matrix_file, 'r') as f:
  data = [yaml.load(line) for line in f.read().rstrip().split('\n')]
datalab.ml.ConfusionMatrix([d['predicted'] for d in data],
                           [d['target'] for d in data],
                           [d['count'] for d in data]).plot()


To generate a pipeline that runs in cloud, simply add --cloud to "%ml evaluate". Also all paths need to be GCS paths.